In [1]:
import time
import numpy as np
from operator import itemgetter
import networkx as nx
import matplotlib.pyplot as plt
from gurobipy import *

# Data

In [2]:
# An array of distances between customers.
dist = np.array([
    [0, 548, 776, 696, 582, 274, 502, 194, 308, 194, 536, 502, 388, 354, 468, 776, 662],
    [548, 0, 684, 308, 194, 502, 730, 354, 696, 742, 1084, 594, 480, 674, 1016, 868, 1210],
    [776, 684, 0, 992, 878, 502, 274, 810, 468, 742, 400, 1278, 1164, 1130, 788, 1552, 754],
    [696, 308, 992, 0, 114, 650, 878, 502, 844, 890, 1232, 514, 628, 822, 1164, 560, 1358],
    [582, 194, 878, 114, 0, 536, 764, 388, 730, 776, 1118, 400, 514, 708, 1050, 674, 1244],
    [274, 502, 502, 650, 536, 0, 228, 308, 194, 240, 582, 776, 662, 628, 514, 1050, 708],
    [502, 730, 274, 878, 764, 228, 0, 536, 194, 468, 354, 1004, 890, 856, 514, 1278, 480],
    [194, 354, 810, 502, 388, 308, 536, 0, 342, 388, 730, 468, 354, 320, 662, 742, 856],
    [308, 696, 468, 844, 730, 194, 194, 342, 0, 274, 388, 810, 696, 662, 320, 1084, 514],
    [194, 742, 742, 890, 776, 240, 468, 388, 274, 0, 342, 536, 422, 388, 274, 810, 468],
    [536, 1084, 400, 1232, 1118, 582, 354, 730, 388, 342, 0, 878, 764, 730, 388, 1152, 354],
    [502, 594, 1278, 514, 400, 776, 1004, 468, 810, 536, 878, 0, 114, 308, 650, 274, 844],
    [388, 480, 1164, 628, 514, 662, 890, 354, 696, 422, 764, 114, 0, 194, 536, 388, 730],
    [354, 674, 1130, 822, 708, 628, 856, 320, 662, 388, 730, 308, 194, 0, 342, 422, 536],
    [468, 1016, 788, 1164, 1050, 514, 514, 662, 320, 274, 388, 650, 536, 342, 0, 764, 194],
    [776, 868, 1552, 560, 674, 1050, 1278, 742, 1084, 810, 1152, 274, 388, 422, 764, 0, 798],
    [662, 1210, 754, 1358, 1244, 708, 480, 856, 514, 468, 354, 844, 730, 536, 194, 798, 0]
])

# Each location has a demand corresponding to the quantity—for example, weight
# or volume—of the item to be delivered.
dem = np.array([0, 1, 1, 2, 4, 2, 4, 8, 8, 1, 2, 1, 2, 4, 4, 8, 8])

# The number of vehicles in the fleet.
n_vehicles = 5

# Each vehicle has a capacity: the maximum quantity that it can hold.
cap = 15

In [3]:
assert dist.shape[0]==dist.shape[1]==len(dem), 'Check input data dimensions'
assert dem[0]==0, 'Check demand data'

In [4]:
n_customers = len(dem)-1

# SAVINGS ALGORITHM with NetworkX

In [5]:
# Log time
savings_start = time.time()

### 0 - Initialize

#### Create empty graph for savings algorithm

In [6]:
# Create graph
G = nx.DiGraph()

# Add depot vertex
G.add_node(0)

# Add customer vertices
G.add_nodes_from([i for i in range(1, n_customers+1)])

#### One route per customer

In [7]:
n_routes = n_customers

In [8]:
# For every customer, create a route from depot
G.add_edges_from([(0, i, {'weight': dist[0, i]}) for i in range(1, n_customers+1)])
G.add_edges_from([(i, 0, {'weight': dist[i, 0]}) for i in range(1, n_customers+1)])

In [9]:
# pos = nx.spring_layout(G)
# nx.draw(G, pos, font_size=16, with_labels=False)
# nx.draw_networkx_labels(G, pos)
# labels = nx.get_edge_attributes(G, 'weight')
# nx.draw_networkx_edge_labels(G, pos, edge_labels=labels)
# plt.show()

### 1 - Create savings list

In [10]:
savings = []

for i in range(1, n_customers+1):
    for j in range(i+1, n_customers+1):
        if i!=j:
            temp = dist[i][0]+dist[0][j]-dist[i][j]
            if temp > 0:
                savings.append( ((i, j), temp) )

### 2 - Sort savings list in in descending order

In [11]:
savings = sorted(savings, key=itemgetter(1), reverse=True)

In [12]:
# for i, sav in enumerate(savings):
#     print(i, '--', sav)

### 3 - Merge routes

In [13]:
for i in range(len(savings)):
    sav = savings[i][0]
    
    # 1. i & j are not in same route
    condition_1 = True

    for route in nx.simple_cycles(G):
        route.append(0)
        if (sav[0] in route) & (sav[1] in route):
            condition_1 = False
            break
    
    # 2. neither i or j are interior of an existing route
    # (both notes are still directly connected to the depot on their respective routes)
    condition_2a = False

    for route in nx.simple_cycles(G):
        route.append(0)
        if (route[1]==sav[0]):
            condition_2a = True
            break
    
    condition_2b = False
    
    for route in nx.simple_cycles(G):
        route.append(0)
        if (route[-2]!=sav[1]):
            condition_2b = True
            break
    
    # 3. The vehicle capacity constraints are not violated by the merged route.
    condition_3 = True

    for route in nx.simple_cycles(G):
        if sav[0] in route:
            sav0_route_dem = (sum( dem[route] ))
        if sav[1] in route:
            sav1_route_dem = (sum( dem[route] ))

    if sav0_route_dem + sav1_route_dem > cap:
        condition_3 = False
    
    # If all above conditions above satisfied, then merge the routes
    
    if condition_1 & (condition_2a & condition_2b) & condition_3:        
        if G.has_edge(sav[0], 0) & G.has_edge(0, sav[1]):
            G.remove_edge(sav[0], 0)
            G.remove_edge(0, sav[1])
            G.add_edge(sav[0], sav[1], weight=dist[sav[0]][sav[1]])
            n_routes -= 1
            
#             ##
#             print(i, '--', sav)
#             pos = nx.spring_layout(G)
#             nx.draw(G, pos, font_size=16, with_labels=False)
#             nx.draw_networkx_labels(G, pos)
#             labels = nx.get_edge_attributes(G,'cost')
#             nx.draw_networkx_edge_labels(G, pos, edge_labels=labels)
#             plt.show()
#             ##
            
        elif G.has_edge(0, sav[0]) & G.has_edge(sav[1], 0):
            G.remove_edge(0, sav[0])
            G.remove_edge(sav[1], 0)
            G.add_edge(sav[1], sav[0], cost=dist[sav[1]][sav[0]])
            n_routes -= 1
            
#             ##
#             print(i, '--', sav)
#             pos = nx.spring_layout(G)
#             nx.draw(G, pos, font_size=16, with_labels=False)
#             nx.draw_networkx_labels(G, pos)
#             labels = nx.get_edge_attributes(G,'cost')
#             nx.draw_networkx_edge_labels(G, pos, edge_labels=labels)
#             plt.show()
#             ##

In [14]:
# Show time
print("--- %s seconds ---" % (time.time() - savings_start))

--- 0.28806090354919434 seconds ---


In [15]:
for route in nx.simple_cycles(G):
    route.append(0)
    print(dem[route].sum(), route)

12 [0, 14, 16, 0]
15 [0, 12, 11, 15, 13, 0]
10 [0, 9, 10, 2, 6, 5, 0]
8 [0, 8, 0]
15 [0, 7, 1, 3, 4, 0]


# COLUMN GENERATION with Gurobi

### MIP input data from initial solution

In [16]:
routes = []

a = np.zeros((n_customers, n_routes))
# if vertex i in route k then a_ik=1
for k, route in enumerate(nx.simple_cycles(G)):
    routes.append(route)
    for i in range(1, n_customers+1):
        if i in route:
            a[i-1][k] = 1
            
b = np.zeros((n_customers+1, n_customers+1, n_routes))
# if (i, j) in route k then b_ijk=1
for k, route in enumerate(routes):
    route.append(0)
    for r in range(len(route)-1):
        b[route[r]][route[r+1]][k] = 1
    route.pop()
    
c = np.zeros(n_routes)
# compute cost of every route
for k, route in enumerate(routes):
    route.append(0)
    for r in range(len(route)-1):
        c[k] += (b[route[r]][route[r+1]][k]*dist[route[r]][route[r+1]])
    route.pop()

### Initial Master Problem

In [17]:
cg_start = time.time()

In [18]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#     INITIAL MASTER PROBLEM
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# Initialize master problem instance
master = Model('master_problem')

# Add variables
y = master.addVars(n_routes, obj=c, vtype=GRB.CONTINUOUS, name='y')
master.update()

# Add constraints
master.addConstr((quicksum(y[k] for k in range(n_routes)) <= n_vehicles), name='fleet_size_constr')
master.addConstrs((quicksum(a[i][k]*y[k] for k in range(n_routes)) >= 1 for i in range(n_customers)),
                  name='set_covering_constr')

# Set objective
master.ModelSense=GRB.MINIMIZE

# Gurobi params
master.setParam('LogToConsole', 0)

# Solve initial bfs 'y'
master.optimize()

Academic license - for non-commercial use only


### Subtour Elimination constraints for Pricing Problem to be added through a callback

In [19]:
def subtour_elim(model, where):
    
    # Found a new (integer) solution to pricing problem problem, so use callback
    if where == GRB.callback.MIPSOL:
        
        # Extract 'x' values from solution
        sol = np.array(model.cbGetSolution(model._vars)).reshape(n_customers+2, n_customers+2)
        sol = np.where(sol>0.5, 1, 0) # Account for solver Numerical stability
        G_sol = nx.DiGraph(sol)
        
        for route in nx.simple_cycles(G_sol):
            route.append(route[0])
            subtour = route

            # Set-up subtour elimination constraint
            expr = LinExpr()
            for i in range(len(subtour)-1):
                expr += model._vars[(n_customers+2)*subtour[i] + subtour[i+1]]
            model.cbLazy(expr <= len(subtour)-2)

### Modify data for Pricing Problem

In [20]:
# Delinking depot-out and depot-in
# Depot becomes depot-out & add one more node for depot-in

# Prohibitive distance between:
# 1. depot-in and depot-out
# 2. customer vertices and depot-out
# 3. depot-in and customer vertices

dist_2 = np.pad(dist, ((0,1), (0,1)), mode='constant', constant_values=9999999)

for i in range(1, n_customers+1):
    dist_2[i,n_customers+1] = dist[i,0]
    dist_2[i,0] = 9999999

# Distance from depot-in to itself is zero
dist_2[n_customers+1,n_customers+1] = 0

# Assign demand=0 to depot-in
dem_2 = np.append(dem, dem[0])

### Column Generation through Pricing Problem

In [21]:
print('Iteration 0:', '\nObjective Value:', round(master.ObjVal, 2))

for iter_ in range(1, 100):
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    #     COLUMN GENERATION: PRICING PROBLEM
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

    # Dual variable values corresponding to fleet_size_constr & set_covering_constr in primal
    # (use in calculating reduced cost of arcs for pricing problem)
    w = master.getAttr('pi')
    w.append(w[0])

    # Initialize pricing problem instance
    pricing = Model('pricing_problem')

    # Add binary variables
    x = pricing.addVars(n_customers+2, n_customers+2, vtype=GRB.BINARY, name='x')
    pricing.update()

    # Add constraints
    pricing.addConstr(quicksum(dem_2[i]*x.sum(i,'*') for i in range(n_customers+2)) <= cap, name='capacity_constr')
    pricing.addConstr((quicksum(x[i,n_customers+1] for i in range(n_customers+1)) == 1), name='flow_in_constr')
    pricing.addConstr((quicksum(x[0,j] for j in range(1, n_customers+2)) == 1), name='flow_out_constr')
    pricing.addConstrs((x.sum('*',k) == x.sum(k,'*') for k in range(1, n_customers+1)), name='flow_bal_constr_'+str(k))
    pricing.addConstrs((x[i,i] == 0 for i in range(n_customers+2)), name='prevent_self_loop_constr')

    # Initialize objective function
    obj_pricing = LinExpr()
    for i in range(n_customers+2):
        for j in range(n_customers+2):
            # Reduced cost
            obj_pricing += (dist_2[i,j]-w[i])*x[i,j]

    # Set objective
    pricing.setObjective(obj_pricing, GRB.MINIMIZE)

    # Gurobi params
    pricing.setParam('OutputFlag', 0)
    pricing.setParam('LazyConstraints', 1)

    # Solve pricing problem
    pricing._vars = pricing.getVars()
    pricing.optimize(subtour_elim)

    # Status update
    print('Pricing problem:', round(pricing.ObjVal, 2))

    # End Column Generation if no column with neg. reduced cost found
    if pricing.ObjVal >= 0:
        print('\nDone!\n')
        print("--- %s seconds ---" % (time.time() - cg_start))
        break

    # Extract solution from pricing problem in form of a directed graph
    sol = np.array(pricing.X).reshape(n_customers+2, n_customers+2)
    sol = np.where(sol>0.5, 1, 0)  # Account for solver Numerical stability
    G_sol = nx.DiGraph(sol)

    # Extract resource-constraind elementary shortest path (rcesp) from the solution graph
    rcesp = nx.shortest_path(G_sol, source=0, target=n_customers+1)
    rcesp.pop()
    
    # Check if rcesp already in routes we have
    condition_new_route = True
    for route in routes:
        if route == rcesp:
            condition_new_route = False
            break
    
    # If rcesp not found in already present routes, then add to master problem and solve again
    if condition_new_route:
        
        # Status update
        print(dem[rcesp].sum(), route)
        
        # Add route
        routes.append(rcesp)
        n_routes += 1

        # Revise input data
        
        a = np.zeros((n_customers, n_routes))
        # if vertex i in route k then a_ik=1
        for k, route in enumerate(routes):
            for i in range(1, n_customers+1):
                if i in route:
                    a[i-1][k] = 1
        
        b = np.zeros((n_customers+1, n_customers+1, n_routes))
        # if (i, j) in route k then b_ijk=1
        for k, route in enumerate(routes):
            route.append(0)
            for r in range(len(route)-1):
                b[route[r]][route[r+1]][k] = 1
            route.pop()
        
        c = np.zeros(n_routes)
        for k, route in enumerate(routes):
            route.append(0)
            for r in range(len(route)-1):
                c[k] += (b[route[r]][route[r+1]][k]*dist[route[r]][route[r+1]])
            route.pop()
        
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        #     REVISED MASTER PROBLEM
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        
        # Reset
        master.reset(0)
        
        # Initialize master problem instance
        master = Model('master_problem')

        # Add variables
        y = master.addVars(n_routes, obj=c, vtype=GRB.CONTINUOUS, name='y')
        master.update()

        # Add constraints
        master.addConstr((quicksum(y[k] for k in range(n_routes)) <= n_vehicles), name='fleet_size_constr')
        master.addConstrs((quicksum(a[i][k]*y[k] for k in range(n_routes)) >= 1 for i in range(n_customers)),
                          name='set_covering_constr')

        # Set objective
        master.ModelSense=GRB.MINIMIZE
        
        # Gurobi params
        master.setParam('LogToConsole', 0)
        
        # Solve initial bfs 'y'
        master.optimize()
        print('\nIteration:', iter_, '\nObjective Value:', round(master.ObjVal, 2))
        
        # Reset
        pricing.reset(0)

Iteration 0: 
Objective Value: 6756.0
Pricing problem: -3720.0
15 [0, 7, 1, 3, 4]

Iteration: 1 
Objective Value: 6756.0
Pricing problem: -3264.0
14 [0, 14, 8, 2, 1, 11]

Iteration: 2 
Objective Value: 6756.0
Pricing problem: -2488.0
12 [0, 12, 3, 5, 8]

Iteration: 3 
Objective Value: 6756.0
Pricing problem: -2488.0
9 [0, 13, 4, 6]

Iteration: 4 
Objective Value: 6756.0
Pricing problem: -1940.0
12 [0, 7, 9]

Iteration: 5 
Objective Value: 6756.0
Pricing problem: -1756.0
15 [0, 10, 16, 9, 1]

Iteration: 6 
Objective Value: 6756.0
Pricing problem: -2636.0
13 [0, 15, 4, 1, 5]

Iteration: 7 
Objective Value: 6756.0
Pricing problem: -2556.0
15 [0, 12, 1, 2, 16, 9]

Iteration: 8 
Objective Value: 6756.0
Pricing problem: -1331.33
14 [0, 8, 5, 1, 13]

Iteration: 9 
Objective Value: 6756.0
Pricing problem: -2054.0
15 [0, 15, 3, 1, 2, 5]

Iteration: 10 
Objective Value: 6756.0
Pricing problem: -1257.45
14 [0, 12, 1, 6, 8]

Iteration: 11 
Objective Value: 6756.0
Pricing problem: -1124.0
15 [0, 5,

### View solution

In [22]:
for v in master.getVars():
    if v.X > 0.5:
        print('%g %s' % (v.X, v.varName))

1 y[1]
1 y[4]
1 y[23]
1 y[26]


In [23]:
print(
    '',
    dem[routes[1]].sum(), routes[1], '\n',
    dem[routes[4]].sum(), routes[4], '\n',
    dem[routes[23]].sum(), routes[23], '\n',
    dem[routes[26]].sum(), routes[26]
)

 15 [0, 12, 11, 15, 13] 
 15 [0, 7, 1, 3, 4] 
 15 [0, 5, 2, 6, 8] 
 15 [0, 9, 10, 16, 14]


### Column Generation returns the integer optimal solution for this instance within 35 seconds.